In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# CONFIG
VECTORIZER_PICKLE = 'vectorizer.pkl'
CLUSTERING_PICKLE = 'clustering.pkl'
INDUSTRIES_LIST = '../industries.txt'

unlabeled_data = '../unlabeled.csv'

data_types = {
    'id': np.int64,
    'text': str,
}

In [2]:
%%time
print('Reading data')
data = pd.read_csv(unlabeled_data, usecols=['id', 'text'], dtype=data_types, engine='c', index_col='id')
data.text.replace(np.nan, "", inplace=True)
print('Number of samples texts: ', len(data))

Reading data
Number of samples texts:  200000
CPU times: user 9.06 s, sys: 1.67 s, total: 10.7 s
Wall time: 12.8 s


In [3]:
%%time
vectorizer = pickle.load(open(VECTORIZER_PICKLE, 'rb'))

CPU times: user 4.34 s, sys: 1.47 s, total: 5.81 s
Wall time: 22.9 s


In [4]:
%%time
clustering = pickle.load(open(CLUSTERING_PICKLE, 'rb'))

CPU times: user 13.4 ms, sys: 1.23 ms, total: 14.6 ms
Wall time: 35 ms


In [ ]:
data['word_vec'] = data.text.apply(lambda x: vectorizer.transform([x]))
data.head(10)

<1x67092 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>